# MLflow Validation Test - Self-Contained Model Registration

This notebook generates synthetic data and registers a simple model to validate our MLflow infrastructure.

In [ ]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

print("Libraries imported successfully")
print(f"MLflow tracking URI: {mlflow.get_tracking_uri()}")

In [ ]:
# Generate synthetic sensor data
np.random.seed(42)

# Create 1000 samples with 5 features
n_samples = 1000
n_features = 5

# Normal operation data (90% of samples)
normal_data = np.random.normal(0, 1, (int(n_samples * 0.9), n_features))

# Anomalous data (10% of samples)
anomalous_data = np.random.normal(3, 1.5, (int(n_samples * 0.1), n_features))

# Combine data
X_synthetic = np.vstack([normal_data, anomalous_data])
y_synthetic = np.hstack([np.zeros(len(normal_data)), np.ones(len(anomalous_data))])

# Create feature names
feature_names = [f'feature_{i}' for i in range(n_features)]

print(f"Generated {len(X_synthetic)} synthetic samples")
print(f"Features: {feature_names}")
print(f"Normal samples: {len(normal_data)}, Anomalous samples: {len(anomalous_data)}")

In [ ]:
# Configure MLflow
mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("Synthetic_Data_Validation")

print(f"MLflow configured to: {mlflow.get_tracking_uri()}")
print(f"Active experiment: {mlflow.get_experiment_by_name('Synthetic_Data_Validation')}")

In [ ]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_synthetic, y_synthetic, test_size=0.3, random_state=42, stratify=y_synthetic
)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train_scaled.shape}")
print(f"Test set: {X_test_scaled.shape}")

In [ ]:
# Train and register model with MLflow
with mlflow.start_run(run_name="Synthetic_Validation_IsolationForest") as run:
    # Log parameters
    contamination = 0.1
    mlflow.log_param("model_type", "IsolationForest")
    mlflow.log_param("contamination", contamination)
    mlflow.log_param("n_samples", n_samples)
    mlflow.log_param("n_features", n_features)
    mlflow.log_param("data_type", "synthetic")
    
    # Train model
    model = IsolationForest(contamination=contamination, random_state=42)
    model.fit(X_train_scaled)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_binary = (y_pred == -1).astype(int)  # Convert to binary (1 for anomaly)
    
    # Calculate basic metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score
    
    accuracy = accuracy_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary, zero_division=0)
    recall = recall_score(y_test, y_pred_binary, zero_division=0)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Log feature names as artifact
    with open("/tmp/feature_names_synthetic.txt", "w") as f:
        f.write("\n".join(feature_names))
    mlflow.log_artifact("/tmp/feature_names_synthetic.txt")
    
    # Register the model
    model_name = "synthetic_validation_isolation_forest"
    mlflow.sklearn.log_model(
        model, 
        "model",
        registered_model_name=model_name
    )
    
    print(f"Model registered as: {model_name}")
    print(f"Run ID: {run.info.run_id}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

In [ ]:
# Test model loading from MLflow
print("\n=== Testing Model Loading ===")
try:
    # Load the model back from MLflow
    loaded_model = mlflow.sklearn.load_model(f"models:/{model_name}/latest")
    
    # Test prediction with the loaded model
    test_sample = X_test_scaled[:5]  # Test with first 5 samples
    predictions = loaded_model.predict(test_sample)
    
    print(f"Successfully loaded model: {model_name}")
    print(f"Test predictions: {predictions}")
    print("✅ MLflow model loading validation PASSED")
    
except Exception as e:
    print(f"❌ MLflow model loading validation FAILED: {e}")
    raise

In [ ]:
print("\n=== Synthetic Data Validation Summary ===")
print("✅ Synthetic data generation: SUCCESS")
print("✅ Model training: SUCCESS")
print("✅ MLflow logging: SUCCESS")
print("✅ Model registration: SUCCESS")
print("✅ Model loading validation: SUCCESS")
print("\n🎉 All validation tests passed! MLflow infrastructure is working correctly.")